In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="1"
os.environ["XLA_FLAGS"] = "--xla_gpu_cuda_data_dir='/usr/lib/cuda'"

In [ ]:
import tensorflow as tf

# Check available GPUs
print("Available GPUs:", tf.config.list_physical_devices('GPU'))

# Force TensorFlow to use GPU:1
with tf.device('/GPU:1'):
    print("Running on GPU:1")


In [ ]:
import os
import matplotlib.pyplot as plt
import copy
import random
import numpy as np
import pandas as pd
import tensorflow as tf

from tqdm import tqdm
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification,DataCollatorWithPadding
from datasets import load_dataset
from tensorflow.keras.optimizers.schedules import PolynomialDecay
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import Adam,Nadam
from datasets import Dataset, DatasetDict
from tensorflow.keras.layers import Dense, BatchNormalization
from transformers import DataCollatorWithPadding
import pandas as pd
from nrclex import NRCLex
from nltk import download
from datasets import load_dataset
from datasets import DatasetDict
from sklearn.model_selection import train_test_split
from collections import Counter
os.environ["TOKENIZERS_PARALLELISM"] = "false"
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
from tensorflow.keras.layers import Dropout, Dense, Reshape, Bidirectional, GRU, GlobalMaxPooling1D,Flatten,Reshape,MultiHeadAttention,Layer,Activation, Multiply, Lambda, Permute
from tensorflow.keras import Input, Model, regularizers
from datasets import load_dataset
from transformers import AutoTokenizer

model_checkpoint = "roberta-large"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

from transformers import TFAutoModelForSequenceClassification
import tensorflow as tf
from tensorflow.keras.layers import Dropout, Dense, Reshape, Bidirectional, GRU, GlobalMaxPooling1D,Flatten,Reshape
from tensorflow.keras import Input, Model
from tensorflow.keras.optimizers import Nadam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.layers import Activation, Multiply, Lambda, Permute
from tensorflow.keras import regularizers

# Define the BERT model
#model_checkpoint = "bert-base-cased"
model = TFAutoModelForSequenceClassification.from_pretrained(model_checkpoint)
roberta_model=model.roberta

from tensorflow.keras.layers import Layer
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.layers import Layer, Input
from tensorflow.keras import initializers, regularizers, constraints

REMOVE_FACTOR = -10000

In [ ]:
model_checkpoint = "roberta-large"  # Use RoBERTa model
batch_size = 8
batch_size = 32
max_length = 200
model_checkpoint2 = "bert-base-cased"
model_checkpoint2 = "bert-base-cased"
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint2)
labels=  ['against','favor','none']
max_length = 200

In [ ]:
def get_top_emotions(text):
    # Analyze sentiment using NRCLex
    text_obj = NRCLex(text)
    # Get sentiment scores
    sentiment_scores = text_obj.affect_frequencies

    # Sort emotions by scores in descending order
    sorted_emotions = sorted(sentiment_scores.items(), key=lambda x: x[1], reverse=True)

    # Get the top 3 emotions
    top_emotions = [emotion for emotion, _ in sorted_emotions[:3]]

    return top_emotions

def get_emotions(row):
    # Get top emotions for source text and reply text
    source_top_emotions = get_top_emotions(row['Target'])
    reply_top_emotions = get_top_emotions(row['Tweet'])
    
    # Separate each emotion into its own column
    source_emotion_columns = {
        f"source_emotion_{i+1}": source_top_emotions[i] if i < len(source_top_emotions) else None
        for i in range(3)
    }
    reply_emotion_columns = {
        f"reply_emotion_{i+1}": reply_top_emotions[i] if i < len(reply_top_emotions) else None
        for i in range(3)
    }
    
    return {**source_emotion_columns, **reply_emotion_columns}


train_df = pd.read_csv("semeval_train.csv")
valid_df = pd.read_csv("semeval_test.csv")
test_df = pd.read_csv("semeval_test.csv")

dataset = DatasetDict({
    "train": Dataset.from_pandas(train_df),
    "valid": Dataset.from_pandas(valid_df),
    "test": Dataset.from_pandas(test_df),
})
dataset = dataset.map(get_emotions)

In [ ]:
# Define a common preprocessing function
def preprocess_function(records, text_column):
    # Tokenize the text
    tokenized = tokenizer(records[text_column], truncation=True, padding='max_length', return_token_type_ids=True, max_length=200)
    return {
        'input_ids': tokenized['input_ids'],
        'attention_mask': tokenized['attention_mask'],

    }

source_text_column = 'Target'
reply_text_column = 'Tweet'
source_emotion_1 = 'source_emotion_1'
source_emotion_2 = 'source_emotion_2'
source_emotion_3 = 'source_emotion_3'

reply_emotion_1 = 'reply_emotion_1'
reply_emotion_2 = 'reply_emotion_2'
reply_emotion_3 = 'reply_emotion_3'

# Preprocess source text
encoded_dataset_source = dataset.map(lambda x: preprocess_function(x, source_text_column), batched=True)

# Preprocess reply text
encoded_dataset_reply = dataset.map(lambda x: preprocess_function(x, reply_text_column), batched=True)

encoded_dataset_source_emotion1 = dataset.map(lambda x: preprocess_function(x, source_emotion_1), batched=True)
encoded_dataset_source_emotion2 = dataset.map(lambda x: preprocess_function(x, source_emotion_2), batched=True)
encoded_dataset_source_emotion3 = dataset.map(lambda x: preprocess_function(x, source_emotion_3), batched=True)


encoded_dataset_reply_emotion1 = dataset.map(lambda x: preprocess_function(x, reply_emotion_1), batched=True)
encoded_dataset_reply_emotion2 = dataset.map(lambda x: preprocess_function(x, reply_emotion_2), batched=True)
encoded_dataset_reply_emotion3 = dataset.map(lambda x: preprocess_function(x, reply_emotion_3), batched=True)



In [ ]:
input_ids_train_source = np.array(encoded_dataset_source['train']['input_ids'])
attention_mask_train_source = np.array(encoded_dataset_source['train']['attention_mask'])
label_train = np.asarray(encoded_dataset_source['train']["labelValue"]).astype(np.float32)
input_ids_train_reply = np.array(encoded_dataset_reply['train']['input_ids'])
attention_mask_train_reply = np.array(encoded_dataset_reply['train']['attention_mask'])
input_ids_train_source_emotion1 = np.array(encoded_dataset_source_emotion1['train']['input_ids'])
attention_mask_train_source_emotion1 = np.array(encoded_dataset_source_emotion1['train']['attention_mask'])
input_ids_train_reply_emotion1 = np.array(encoded_dataset_reply_emotion1['train']['input_ids'])
attention_mask_train_reply_emotion1 = np.array(encoded_dataset_reply_emotion1['train']['attention_mask'])
input_ids_train_source_emotion2 = np.array(encoded_dataset_source_emotion2['train']['input_ids'])
attention_mask_train_source_emotion2= np.array(encoded_dataset_source_emotion2['train']['attention_mask'])
input_ids_train_reply_emotion2 = np.array(encoded_dataset_reply_emotion2['train']['input_ids'])
attention_mask_train_reply_emotion2 = np.array(encoded_dataset_reply_emotion2['train']['attention_mask'])
input_ids_train_source_emotion3 = np.array(encoded_dataset_source_emotion3['train']['input_ids'])
attention_mask_train_source_emotion3 = np.array(encoded_dataset_source_emotion3['train']['attention_mask'])
input_ids_train_reply_emotion3 = np.array(encoded_dataset_reply_emotion3['train']['input_ids'])
attention_mask_train_reply_emotion3 = np.array(encoded_dataset_reply_emotion3['train']['attention_mask'])

In [ ]:
input_ids_valid_source = np.array(encoded_dataset_source['valid']['input_ids'])
attention_mask_valid_source = np.array(encoded_dataset_source['valid']['attention_mask'])
label_val = np.asarray(encoded_dataset_source['valid']["labelValue"]).astype(np.float32)
input_ids_valid_reply = np.array(encoded_dataset_reply['valid']['input_ids'])
attention_mask_valid_reply = np.array(encoded_dataset_reply['valid']['attention_mask'])
input_ids_valid_source_emotion1 = np.array(encoded_dataset_source_emotion1['valid']['input_ids'])
attention_mask_valid_source_emotion1 = np.array(encoded_dataset_source_emotion1['valid']['attention_mask'])
input_ids_valid_reply_emotion1 = np.array(encoded_dataset_reply_emotion1['valid']['input_ids'])
attention_mask_valid_reply_emotion1 = np.array(encoded_dataset_reply_emotion1['valid']['attention_mask'])
input_ids_valid_source_emotion2 = np.array(encoded_dataset_source_emotion2['valid']['input_ids'])
attention_mask_valid_source_emotion2 = np.array(encoded_dataset_source_emotion2['valid']['attention_mask'])
input_ids_valid_reply_emotion2 = np.array(encoded_dataset_reply_emotion2['valid']['input_ids'])
attention_mask_valid_reply_emotion2 = np.array(encoded_dataset_reply_emotion2['valid']['attention_mask'])
input_ids_valid_source_emotion3 = np.array(encoded_dataset_source_emotion3['valid']['input_ids'])
attention_mask_valid_source_emotion3 = np.array(encoded_dataset_source_emotion3['valid']['attention_mask'])
input_ids_valid_reply_emotion3 = np.array(encoded_dataset_reply_emotion3['valid']['input_ids'])
attention_mask_valid_reply_emotion3 = np.array(encoded_dataset_reply_emotion3['valid']['attention_mask'])


In [ ]:
input_ids_test_source = np.array(encoded_dataset_source['test']['input_ids'])
attention_mask_test_source = np.array(encoded_dataset_source['test']['attention_mask'])
label_test = np.asarray(encoded_dataset_source['test']["labelValue"]).astype(np.float32)
input_ids_test_reply = np.array(encoded_dataset_reply['test']['input_ids'])
attention_mask_test_reply = np.array(encoded_dataset_reply['test']['attention_mask'])
input_ids_test_source_emotion1 = np.array(encoded_dataset_source_emotion1['test']['input_ids'])
attention_mask_test_source_emotion1 = np.array(encoded_dataset_source_emotion1['test']['attention_mask'])
input_ids_test_reply_emotion1 = np.array(encoded_dataset_reply_emotion1['test']['input_ids'])
attention_mask_test_reply_emotion1 = np.array(encoded_dataset_reply_emotion1['test']['attention_mask'])
input_ids_test_source_emotion2 = np.array(encoded_dataset_source_emotion2['test']['input_ids'])
attention_mask_test_source_emotion2 = np.array(encoded_dataset_source_emotion2['test']['attention_mask'])
input_ids_test_reply_emotion2 = np.array(encoded_dataset_reply_emotion2['test']['input_ids'])
attention_mask_test_reply_emotion2 = np.array(encoded_dataset_reply_emotion2['test']['attention_mask'])
input_ids_test_source_emotion3 = np.array(encoded_dataset_source_emotion3['test']['input_ids'])
attention_mask_test_source_emotion3 = np.array(encoded_dataset_source_emotion3['test']['attention_mask'])
input_ids_test_reply_emotion3 = np.array(encoded_dataset_reply_emotion3['test']['input_ids'])
attention_mask_test_reply_emotion3 = np.array(encoded_dataset_reply_emotion3['test']['attention_mask'])

In [ ]:
tokenized_labels=tokenizer(labels,
          return_tensors='tf',
          return_token_type_ids=False,truncation=True,padding='max_length',  max_length=3)


In [ ]:

class BERTWrapperLayer(Layer):
    def __init__(self, bert_model, **kwargs):
        super(BERTWrapperLayer, self).__init__(**kwargs)
        self.bert_model = bert_model

    def call(self, inputs, **kwargs):
        input_ids, attention_mask = inputs
        bert_output = self.bert_model(input_ids, attention_mask=attention_mask)
        return bert_output.last_hidden_state
bert_last_hidden_state_labels = BERTWrapperLayer(roberta_model)([tokenized_labels['input_ids'], tokenized_labels["attention_mask"]])

In [ ]:
arr1= bert_last_hidden_state_labels
arr2 = np.mean(arr1, axis=1)

In [ ]:
input_ids_source = Input(shape=(max_length,), dtype=tf.int32, name="input_ids_source")
attention_mask_source = Input(shape=(max_length,), dtype=tf.int32, name="attention_mask_source")
input_ids_reply = Input(shape=(max_length,), dtype=tf.int32, name="input_ids_reply")
attention_mask_reply = Input(shape=(max_length,), dtype=tf.int32, name="attention_mask_reply")
arr2_input = Input(shape=(3,1024), dtype=tf.float32, name="arr2_input")


# Input layers for emotion 1
input_ids_source_emotion1 = Input(shape=(max_length,), dtype=tf.int32, name="input_ids_source_emotion1")
attention_mask_source_emotion1 = Input(shape=(max_length,), dtype=tf.int32, name="attention_mask_source_emotion1")
input_ids_reply_emotion1 = Input(shape=(max_length,), dtype=tf.int32, name="input_ids_reply_emotion1")
attention_mask_reply_emotion1 = Input(shape=(max_length,), dtype=tf.int32, name="attention_mask_reply_emotion1")

# Input layers for emotion 2
input_ids_source_emotion2 = Input(shape=(max_length,), dtype=tf.int32, name="input_ids_source_emotion2")
attention_mask_source_emotion2 = Input(shape=(max_length,), dtype=tf.int32, name="attention_mask_source_emotion2")
input_ids_reply_emotion2 = Input(shape=(max_length,), dtype=tf.int32, name="input_ids_reply_emotion2")
attention_mask_reply_emotion2 = Input(shape=(max_length,), dtype=tf.int32, name="attention_mask_reply_emotion2")

# Input layers for emotion 3
input_ids_source_emotion3 = Input(shape=(max_length,), dtype=tf.int32, name="input_ids_source_emotion3")
attention_mask_source_emotion3 = Input(shape=(max_length,), dtype=tf.int32, name="attention_mask_source_emotion3")
input_ids_reply_emotion3 = Input(shape=(max_length,), dtype=tf.int32, name="input_ids_reply_emotion3")
attention_mask_reply_emotion3 = Input(shape=(max_length,), dtype=tf.int32, name="attention_mask_reply_emotion3")

model = TFAutoModelForSequenceClassification.from_pretrained(model_checkpoint)
roberta_model=model.roberta

roberta_last_hidden_state_source = BERTWrapperLayer(roberta_model)([input_ids_source, attention_mask_source])
roberta_last_hidden_state_reply = BERTWrapperLayer(roberta_model)([input_ids_reply, attention_mask_reply])

roberta_last_hidden_state_s1 = BERTWrapperLayer(roberta_model)([input_ids_source_emotion1, attention_mask_source_emotion1])[:,0,]
roberta_last_hidden_state_s2 = BERTWrapperLayer(roberta_model)([input_ids_source_emotion2, attention_mask_source_emotion2])[:,0,]
roberta_last_hidden_state_s3 = BERTWrapperLayer(roberta_model)([input_ids_source_emotion3, attention_mask_source_emotion3])[:,0,]
roberta_last_hidden_state_r1 = BERTWrapperLayer(roberta_model)([input_ids_reply_emotion1, attention_mask_reply_emotion1])[:,0,]
roberta_last_hidden_state_r2 = BERTWrapperLayer(roberta_model)([input_ids_reply_emotion2, attention_mask_reply_emotion2])[:,0,]
roberta_last_hidden_state_r3 = BERTWrapperLayer(roberta_model)([input_ids_reply_emotion3, attention_mask_reply_emotion3])[:,0,]

In [ ]:
stacked_tensor_s = tf.stack([roberta_last_hidden_state_s1, roberta_last_hidden_state_s2, roberta_last_hidden_state_s3], axis=1)
stacked_tensor_r = tf.stack([roberta_last_hidden_state_r1, roberta_last_hidden_state_r2, roberta_last_hidden_state_r3], axis=1)

# Calculate the mean along axis 1
mean_tensor_s = tf.reduce_mean(stacked_tensor_s, axis=1)
mean_tensor_r = tf.reduce_mean(stacked_tensor_r, axis=1)

In [ ]:
sliced_cls_source = roberta_last_hidden_state_source[:, 0]
sliced_cls_reply = roberta_last_hidden_state_reply[:, 0]

In [ ]:
difference_cls = tf.abs(sliced_cls_source - sliced_cls_reply)

In [ ]:
normalized_difference = tf.math.l2_normalize(difference_cls, axis=1)

In [ ]:
#cross-attention-1
import tensorflow as tf

class CrossAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, embed_size, num_heads):
        super(CrossAttentionLayer, self).__init__()
        self.num_heads = num_heads
        self.embed_size = embed_size

        # Query, Key, and Value weight matrices
        self.WQS = tf.keras.layers.Dense(embed_size)
        self.WKS = tf.keras.layers.Dense(embed_size)
        self.WVS = tf.keras.layers.Dense(embed_size)

        self.WQR = tf.keras.layers.Dense(embed_size)
        self.WKR = tf.keras.layers.Dense(embed_size)
        self.WVR = tf.keras.layers.Dense(embed_size)

        # Output weight matrix
        self.WOS = tf.keras.layers.Dense(embed_size)
        self.WOR = tf.keras.layers.Dense(embed_size)

    def scaled_dot_product_attention(self, Q, K, V):
        matmul_qk = tf.matmul(Q, K, transpose_b=True)
        dk = tf.cast(tf.shape(K)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
        output = tf.matmul(attention_weights, V)
        return output, attention_weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.embed_size // self.num_heads))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        source, reply = inputs

        query_s = source
        query_r = reply

        value_s = source
        value_r = reply

        key_s = reply
        key_r = source

        batch_size = tf.shape(query_s)[0]

        # Linear projections
        query_s = self.WQS(query_s)
        key_s = self.WKS(key_s)
        value_s = self.WVS(value_s)

        query_r = self.WQR(query_r)
        key_r = self.WKR(key_r)
        value_r = self.WVR(value_r)

        # Split heads
        query_s = self.split_heads(query_s, batch_size)
        key_s = self.split_heads(key_s, batch_size)
        value_s = self.split_heads(value_s, batch_size)

        query_r = self.split_heads(query_r, batch_size)
        key_r = self.split_heads(key_r, batch_size)
        value_r = self.split_heads(value_r, batch_size)

        # Scaled Dot-Product Attention
        scaled_attention_s, attention_weights_s = self.scaled_dot_product_attention(query_s, key_s, value_s)
        #print(scaled_attention_s)
        scaled_attention_r, attention_weights_r = self.scaled_dot_product_attention(query_r, key_r, value_r)

        # Merge heads
        scaled_attention_s = tf.transpose(scaled_attention_s, perm=[0, 2, 1, 3])
        #print(scaled_attention_s)
        concat_attention_s = tf.reshape(scaled_attention_s, (batch_size, seq_len, self.embed_size))
        #print(concat_attention_s )

        scaled_attention_r = tf.transpose(scaled_attention_r, perm=[0, 2, 1, 3])
        concat_attention_r = tf.reshape(scaled_attention_r, (batch_size, seq_len, self.embed_size))

        # Linear projection
        output_s = self.WOS(concat_attention_s)
        output_r = self.WOR(concat_attention_r)

        return output_s, output_r

# Example usage
seq_len = 200
batch_size = 32
embed_size = 1024
num_heads = 8

cross_attention_layer1 = CrossAttentionLayer(embed_size, num_heads)


# Apply cross-attention
cross_attention_source_output1, cross_attention_reply_output1 = cross_attention_layer1([roberta_last_hidden_state_source, roberta_last_hidden_state_reply])

print("Input Query Shape:", roberta_last_hidden_state_source.shape)
print("Output Shape:", cross_attention_source_output1.shape)


In [ ]:
import tensorflow as tf

class SelfAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, embed_size, num_heads):
        super(SelfAttentionLayer, self).__init__()
        self.num_heads = num_heads
        self.embed_size = embed_size

        # Query, Key, and Value weight matrices
        self.WQ = tf.keras.layers.Dense(embed_size)
        self.WK = tf.keras.layers.Dense(embed_size)
        self.WV = tf.keras.layers.Dense(embed_size)

        # Output weight matrix
        self.WO = tf.keras.layers.Dense(embed_size)

    def scaled_dot_product_attention(self, Q, K, V):
        matmul_qk = tf.matmul(Q, K, transpose_b=True)
        dk = tf.cast(tf.shape(K)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
        output = tf.matmul(attention_weights, V)
        return output, attention_weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.embed_size // self.num_heads))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        x = inputs
        batch_size = tf.shape(x)[0]

        # Linear projections
        query = self.WQ(x)
        key = self.WK(x)
        value = self.WV(x)

        # Split heads
        query = self.split_heads(query, batch_size)
        key = self.split_heads(key, batch_size)
        value = self.split_heads(value, batch_size)

        # Scaled Dot-Product Attention
        scaled_attention, attention_weights = self.scaled_dot_product_attention(query, key, value)

        # Merge heads
        scaled_attention = tf.transpose(scaled_attention, perm=[0, 2, 1, 3])
        concat_attention = tf.reshape(scaled_attention, (batch_size, -1, self.embed_size))

        # Linear projection
        output = self.WO(concat_attention)

        return output, attention_weights

# Example usage


self_attention_layer1 = SelfAttentionLayer(embed_size, num_heads)

# Apply self-attention
self_attention_source_output1, _ = self_attention_layer1(cross_attention_source_output1)
self_attention_reply_output1, _ = self_attention_layer1(cross_attention_reply_output1)
print("Input Data Shape:", cross_attention_source_output1.shape)
print("Output Shape:", self_attention_source_output1.shape)

In [ ]:
#self attention-reply
import tensorflow as tf

class CrossAttentionLayer(tf.keras.layers.Layer):
    def __init__(self, embed_size, num_heads):
        super(CrossAttentionLayer, self).__init__()
        self.num_heads = num_heads
        self.embed_size = embed_size

        # Query, Key, and Value weight matrices
        self.WQS = tf.keras.layers.Dense(embed_size)
        self.WKS = tf.keras.layers.Dense(embed_size)
        self.WVS = tf.keras.layers.Dense(embed_size)

        self.WQR = tf.keras.layers.Dense(embed_size)
        self.WKR = tf.keras.layers.Dense(embed_size)
        self.WVR = tf.keras.layers.Dense(embed_size)

        # Output weight matrix
        self.WOS = tf.keras.layers.Dense(embed_size)
        self.WOR = tf.keras.layers.Dense(embed_size)

    def scaled_dot_product_attention(self, Q, K, V):
        matmul_qk = tf.matmul(Q, K, transpose_b=True)
        dk = tf.cast(tf.shape(K)[-1], tf.float32)
        scaled_attention_logits = matmul_qk / tf.math.sqrt(dk)
        attention_weights = tf.nn.softmax(scaled_attention_logits, axis=-1)
        output = tf.matmul(attention_weights, V)
        return output, attention_weights

    def split_heads(self, x, batch_size):
        x = tf.reshape(x, (batch_size, -1, self.num_heads, self.embed_size // self.num_heads))
        return tf.transpose(x, perm=[0, 2, 1, 3])

    def call(self, inputs):
        source, reply = inputs

        query_s = source
        query_r = reply

        value_s = reply
        value_r = source

        key_s = source
        key_r = reply

        batch_size = tf.shape(query_s)[0]

        # Linear projections
        query_s = self.WQS(query_s)
        key_s = self.WKS(key_s)
        value_s = self.WVS(value_s)

        query_r = self.WQR(query_r)
        key_r = self.WKR(key_r)
        value_r = self.WVR(value_r)

        # Split heads
        query_s = self.split_heads(query_s, batch_size)
        key_s = self.split_heads(key_s, batch_size)
        value_s = self.split_heads(value_s, batch_size)

        query_r = self.split_heads(query_r, batch_size)
        key_r = self.split_heads(key_r, batch_size)
        value_r = self.split_heads(value_r, batch_size)

        # Scaled Dot-Product Attention
        scaled_attention_s, attention_weights_s = self.scaled_dot_product_attention(query_s, key_s, value_s)
        #print(scaled_attention_s)
        scaled_attention_r, attention_weights_r = self.scaled_dot_product_attention(query_r, key_r, value_r)

        # Merge heads
        scaled_attention_s = tf.transpose(scaled_attention_s, perm=[0, 2, 1, 3])
        #print(scaled_attention_s)
        concat_attention_s = tf.reshape(scaled_attention_s, (batch_size, seq_len, self.embed_size))
        #print(concat_attention_s )

        scaled_attention_r = tf.transpose(scaled_attention_r, perm=[0, 2, 1, 3])
        concat_attention_r = tf.reshape(scaled_attention_r, (batch_size, seq_len, self.embed_size))

        # Linear projection
        output_s = self.WOS(concat_attention_s)
        output_r = self.WOR(concat_attention_r)

        return output_s, output_r

# Example usage


cross_attention_layer2 = CrossAttentionLayer(embed_size, num_heads)


# Apply cross-attention
cross_attention_source_output2, cross_attention_reply_output2 = cross_attention_layer2([self_attention_source_output1, self_attention_reply_output1])

print("Input Query Shape:", self_attention_source_output1.shape)
print("Output Shape:", cross_attention_source_output2.shape)


In [ ]:
#source
self_attention_layer2 = SelfAttentionLayer(embed_size, num_heads)
# Apply self-attention
self_attention_source_output2, _ = self_attention_layer2(cross_attention_source_output2)
self_attention_reply_output2, _ = self_attention_layer2(cross_attention_reply_output2)
print("Input Data Shape:", cross_attention_source_output2.shape)
print("Output Shape:", self_attention_source_output2.shape)

In [ ]:


def dot_product(x, kernel):

    if K.backend() == 'tensorflow':
        return K.squeeze(K.dot(x, K.expand_dims(kernel)), axis=-1)
    else:
        return K.dot(x, kernel)

class HierarchicalAttentionLayer(Layer):
  
    def __init__(self, W_regularizer=None, u_regularizer=None, b_regularizer=None,
                 W_constraint=None, u_constraint=None, b_constraint=None,
                 bias=True, **kwargs):

        self.supports_masking = True
        self.init = initializers.get('glorot_uniform')

        self.W_regularizer = regularizers.get(W_regularizer)
        self.u_regularizer = regularizers.get(u_regularizer)
        self.b_regularizer = regularizers.get(b_regularizer)

        self.W_constraint = constraints.get(W_constraint)
        self.u_constraint = constraints.get(u_constraint)
        self.b_constraint = constraints.get(b_constraint)

        self.bias = bias
        super(HierarchicalAttentionLayer, self).__init__(**kwargs)

    def build(self, input_shape):
        assert len(input_shape) == 3

        self.W = self.add_weight(shape=(input_shape[-1], input_shape[-1]),
                                 initializer=self.init,
                                 name='{}_W'.format(self.name),
                                 regularizer=self.W_regularizer,
                                 constraint=self.W_constraint)
        if self.bias:
            self.b = self.add_weight(shape=(input_shape[-1],),
                                     initializer='zeros',
                                     name='{}_b'.format(self.name),
                                     regularizer=self.b_regularizer,
                                     constraint=self.b_constraint)

        self.u = self.add_weight(shape=(input_shape[-1],),
                                 initializer=self.init,
                                 name='{}_u'.format(self.name),
                                 regularizer=self.u_regularizer,
                                 constraint=self.u_constraint)

        super(HierarchicalAttentionLayer, self).build(input_shape)

    def get_config(self):
        config = super(HierarchicalAttentionLayer, self).get_config()
        config.update({
            'W_regularizer': regularizers.serialize(self.W_regularizer),
            'u_regularizer': regularizers.serialize(self.u_regularizer),
            'b_regularizer': regularizers.serialize(self.b_regularizer),
            'W_constraint': constraints.serialize(self.W_constraint),
            'u_constraint': constraints.serialize(self.u_constraint),
            'b_constraint': constraints.serialize(self.b_constraint),
            'bias': self.bias,
        })
        return config

    def compute_mask(self, inputs, mask=None):
        return None

    def call(self, x, mask=None):
        uit = dot_product(x, self.W)

        if self.bias:
            uit += self.b

        uit = K.tanh(uit)
        ait = dot_product(uit, self.u)

        a = K.exp(ait)

        if mask is not None:
            a *= K.cast(mask, K.floatx())

        a /= K.cast(K.sum(a, axis=1, keepdims=True) + K.epsilon(), K.floatx())

        a = K.expand_dims(a)
        weighted_input = x * a
        return K.sum(weighted_input, axis=1)

    def compute_output_shape(self, input_shape):
        return input_shape[0], input_shape[-1]

# Example usage
# a = Input(shape=(50, 768))
# output = HierarchicalAttentionLayer()(a)
word_attention_sum_source=HierarchicalAttentionLayer()(self_attention_reply_output2)
word_attention_sum_reply=HierarchicalAttentionLayer()(self_attention_source_output2)

In [ ]:
difference_emo = tf.abs(mean_tensor_s - mean_tensor_r)
#difference_emo = tf.math.l2_normalize(difference_emo, axis=1)


In [ ]:
#concat word attentions with embeddings difference
combined_representation_cross_attention_embedding_diff = tf.concat([word_attention_sum_source, word_attention_sum_reply,normalized_difference,difference_emo],
                                                                    axis=1)
combined_representation_cross_attention_embedding_diff_af_d= tf.keras.layers.Dense(1024, activation='relu')(combined_representation_cross_attention_embedding_diff)
combined_representation_cross_attention_embedding_diff_af_d

In [ ]:
combined_representation_cross_attention_embedding_diff

In [ ]:

arr_0 = tf.expand_dims(arr2[0], axis=0)  # Add a dimension to match the rows
arr_1 = tf.expand_dims(arr2[1], axis=0) 
arr_2 = tf.expand_dims(arr2[2], axis=0) 
#arr_3 = tf.expand_dims(arr2[3], axis=0) 
# Perform the subtraction
result0 = combined_representation_cross_attention_embedding_diff_af_d - arr_0
result1 = combined_representation_cross_attention_embedding_diff_af_d - arr_1
result2 = combined_representation_cross_attention_embedding_diff_af_d - arr_2
#result3 = combined_representation_cross_attention_embedding_diff_af_d - arr_3
dense_layer1 = tf.keras.layers.Dense(units=520, activation='relu')
dense_layer2 = tf.keras.layers.Dense(units=256, activation='relu') # You can adjust desired_units and activation as needed

# Apply the dense layer to your tensor
result0_af_d = dense_layer1(result0)
result0_af_d = dense_layer2(result0_af_d)

result1_af_d = dense_layer1(result1)
result1_af_d = dense_layer2(result1_af_d)

result2_af_d = dense_layer1(result2)
result2_af_d = dense_layer2(result2_af_d)

# result3_af_d = dense_layer1(result3)
# result3_af_d = dense_layer2(result3_af_d)

diff_labels=tf.concat([result0_af_d,result1_af_d,result2_af_d],axis=1)
diff_labels

In [ ]:
model1 = Model(inputs= [input_ids_source,attention_mask_source,input_ids_reply,attention_mask_reply,
                        input_ids_source_emotion1,attention_mask_source_emotion1,input_ids_reply_emotion1,attention_mask_reply_emotion1,
                        input_ids_source_emotion2,attention_mask_source_emotion2,input_ids_reply_emotion2,attention_mask_reply_emotion2,
                        input_ids_source_emotion3,attention_mask_source_emotion3,input_ids_reply_emotion3,attention_mask_reply_emotion3],
               outputs=[combined_representation_cross_attention_embedding_diff])
#dot_product = tf.matmul(modelctR, dense_layer, transpose_b=True)

# Add dropout regularization after the hierarchical attention layer
#combined_representation = tf.concat([model1.output,v_p_s,v_neg_s,v_neu_s,v_p_r,v_neg_r,v_neu_r,clip_source_text,clip_reply_text,dot_product], axis=1)
combined_representation3 = tf.concat([model1.output,diff_labels], axis=1)
combined_representation3

In [ ]:
combined_representation3

In [ ]:
import tensorflow as tf

# Adding dense and dropout layers
x = tf.keras.layers.Dense(2048, activation='relu')(combined_representation3)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(1024, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(512, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.2)(x)

# Output layer with 4 classes
output1 = tf.keras.layers.Dense(3, activation='softmax')(x)

In [ ]:
modified_model = Model(inputs=[model1.input,arr2_input], outputs=output1)

# Define the optimizer
learning_rate = 2e-6
optimizer = Nadam(learning_rate=learning_rate)

# Define the loss function
loss = SparseCategoricalCrossentropy(from_logits=False)
#CategoricalCrossentropy
num_epochs = 30

In [ ]:
# Compile the model
#modified_model.compile(optimizer=optimizer, loss=loss, metrics=["accuracy"])

In [ ]:
replicated_array_train = np.tile(arr2, (len(encoded_dataset_reply["train"]), 1,1))
replicated_array_val = np.tile(arr2, (len(encoded_dataset_reply["valid"]), 1,1))
replicated_array_val.shape

In [ ]:
replicated_array_train.shape

In [ ]:
# from tensorflow.keras.utils import plot_model
# plot_model(model1, to_file='model_plot_top3emo_avg2.png', show_shapes=True, show_layer_names=True)

In [ ]:
from tensorflow.keras.callbacks import ModelCheckpoint

# Define a checkpoint callback
#checkpoint_filepath = 'best_weights_unbalanced'
#model_checkpoint = ModelCheckpoint(checkpoint_filepath, save_best_only=True, monitor='val_accuracy', mode='max', save_format='tf', verbose=1)
import sys
from datetime import datetime


# Open the file with the date and time in the name
file_name = f"output_semeval_no_preprocessing200seq.txt"
sys.stdout = open(file_name, 'w')

import numpy as np
from tensorflow.keras.callbacks import ModelCheckpoint

# Assuming modified_model is already defined and compiled

# Data Preparation
X = [input_ids_train_source, attention_mask_train_source, input_ids_train_reply, attention_mask_train_reply,
     input_ids_train_source_emotion1, attention_mask_train_source_emotion1, input_ids_train_reply_emotion1, attention_mask_train_reply_emotion1,
     input_ids_train_source_emotion2, attention_mask_train_source_emotion2, input_ids_train_reply_emotion2, attention_mask_train_reply_emotion2,
     input_ids_train_source_emotion3, attention_mask_train_source_emotion3, input_ids_train_reply_emotion3, attention_mask_train_reply_emotion3,     
     replicated_array_train]

label_train = np.asarray(encoded_dataset_source['train']["labelValue"]).astype(np.float32)
label_val = np.asarray(encoded_dataset_source['valid']["labelValue"]).astype(np.float32)

# Define the checkpoint callback to save the model with the highest validation accuracy
# checkpoint_callback = ModelCheckpoint(
#     filepath='final_model_more_dense_layers', # Path where the model will be saved
#     monitor='val_accuracy',   # Metric to monitor
#     save_best_only=True,      # Save only the best model
#     mode='max',               # We want to maximize the monitored metric (accuracy)
#     verbose=1                 # Print messages when saving the model
# )

# Train the model with the checkpoint callback
# history = modified_model.fit(
#     X, label_train,
#     validation_data=(
#         [input_ids_valid_source, attention_mask_valid_source, input_ids_valid_reply, attention_mask_valid_reply,
#          input_ids_valid_source_emotion1, attention_mask_valid_source_emotion1, input_ids_valid_reply_emotion1, attention_mask_valid_reply_emotion1,
#          input_ids_valid_source_emotion2, attention_mask_valid_source_emotion2, input_ids_valid_reply_emotion2, attention_mask_valid_reply_emotion2,
#          input_ids_valid_source_emotion3, attention_mask_valid_source_emotion3, input_ids_valid_reply_emotion3, attention_mask_valid_reply_emotion3,
#          replicated_array_val], 
#         label_val
#     ),
#     epochs=num_epochs, 
#     batch_size=8,
#     callbacks=[checkpoint_callback] # Add the callback here
# )

# Save the final model (if needed)
#modified_model.save('final_model_more_dense_layers')


In [ ]:
replicated_array_test = np.tile(arr2, (len(encoded_dataset_reply["test"]), 1,1))
replicated_array_test.shape

In [ ]:
input_ids_test_source.shape

In [ ]:
# Evaluate the model on the test dataset
from keras.models import load_model

#X_test = [input_ids_test_source,token_type_ids_test_source, attention_mask_test_source,input_ids_test_reply,token_type_ids_test_reply, attention_mask_test_reply,v_p_s_test,v_neg_s_test,v_neu_s_test,v_p_r_test,v_neg_r_test,v_neu_r_test,clip_s_test,clip_r_test,replicated_array_test]
#checkpoint_filepath='modified_model_saved_top3emo_avg'
#modified_model.load_weights(checkpoint_filepath)
modified_model = load_model("final_model_more_dense_layers")

X_test = [input_ids_test_source, attention_mask_test_source,input_ids_test_reply, attention_mask_test_reply,
          input_ids_test_source_emotion1,attention_mask_test_source_emotion1,input_ids_test_reply_emotion1,attention_mask_test_reply_emotion1,
     input_ids_test_source_emotion2,attention_mask_test_source_emotion2,input_ids_test_reply_emotion2,attention_mask_test_reply_emotion2,
     input_ids_test_source_emotion3,attention_mask_test_source_emotion3,input_ids_test_reply_emotion3,attention_mask_test_reply_emotion3,          
          replicated_array_test]
label_test = np.asarray(encoded_dataset_source['test']["labelValue"]).astype(np.float32)

test_results = modified_model.evaluate(X_test,label_test)

# Print the test accuracy
test_accuracy = test_results[1] * 100
print("Test Accuracy: {:.2f}%".format(test_accuracy))

In [ ]:
from sklearn.metrics import f1_score
from sklearn.metrics import precision_recall_fscore_support

# Make predictions on the test dataset
y_pred = modified_model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)  # Convert predicted probabilities to class labels

# Get the true labels from the test dataset
# y_true = tf_test_dataset.map(lambda x, y: y)
# y_true = np.concatenate(list(y_true.as_numpy_iterator()))

# Calculate the F1 score
f1score = f1_score(label_test, y_pred, average='weighted')
f1_score_macro =f1_score(label_test, y_pred, average='macro')

print("F1 Score: {:.2f}".format(f1score))
print("F1 macro Score: {:.2f}",(f1_score_macro))

In [ ]:
import numpy as np
from sklearn.metrics import f1_score

# Make predictions on the test dataset
y_pred = modified_model.predict(X_test)
y_pred = np.argmax(y_pred, axis=1)  # Convert predicted probabilities to class labels

y_true = label_test
num_labels = 3 # Number of labels in the dataset
f1_scores = []
for label in range(num_labels):
    label_true = (y_true == label)
    label_pred = (y_pred == label)
    label_f1_score = f1_score(label_true, label_pred)
    f1_scores.append(label_f1_score)

# Print the F1 score for each label
for label, f1_score in enumerate(f1_scores):
    print("F1 Score for label {}: {:.2f}".format(label, f1_score))

In [ ]:
# support 0.40 deny 0.55 comment 0.90 query 0.22 0.385
# support 0.02 commnet 0.34 deny 0.91 query 0.48
# 'support': 0, 'comment': 1, 'deny': 2, 'query': 3}

In [ ]:
predictions=modified_model.predict(X_test)

In [ ]:
predictions

In [ ]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Assuming y_true and y_pred are NumPy arrays
# If they are not NumPy arrays, you can convert them using np.array(your_list)

# Example true labels (ground truth) for a multiclass problem
y_true = np.array(label_test)

# Example predicted probabilities from your model
# This assumes you have a model predicting probabilities for each class
y_probs = np.array(predictions)

# Convert predicted probabilities to class labels
y_pred = np.argmax(y_probs, axis=-1)

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy}')

# Calculate precision, recall, and F1-score
precision = precision_score(y_true, y_pred, average='weighted')
recall = recall_score(y_true, y_pred, average='weighted')
f1 = f1_score(y_true, y_pred, average='weighted')

print(f'Precision: {precision}')
print(f'Recall: {recall}')
print(f'F1-Score: {f1}')


In [ ]:
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score

# Assuming y_true and y_pred are NumPy arrays
# If they are not NumPy arrays, you can convert them using np.array(your_list)

# Example true labels (ground truth) for a multiclass problem
y_true = np.array(label_test)

# Example predicted probabilities from your model
# This assumes you have a model predicting probabilities for each class
y_probs = np.array(predictions)

# Convert predicted probabilities to class labels
y_pred = np.argmax(y_probs, axis=-1)

# Calculate accuracy
accuracy = accuracy_score(y_true, y_pred)
print(f'Accuracy: {accuracy}')

# Calculate precision, recall, and F1-score
precision = precision_score(y_true, y_pred, average='macro')
recall = recall_score(y_true, y_pred, average='macro')
f1 = f1_score(y_true, y_pred, average='macro')

print(f'Precision macro: {precision}')
print(f'Recall macro: {recall}')
print(f'F1-Score macro: {f1}')
